# Download data

In [1]:
import os
import requests
import zipfile

In [2]:
def ensure_directory_exists(directory):
    """Ensure the directory exists. Create it if not."""
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"Directory '{directory}' created.")
        return False
    else:
        print(f"Directory '{directory}' already exists.")
        return True

In [3]:
def download_file_if_not_exists(file_path, url):
    """Download a file if it does not exist."""
    if not os.path.exists(file_path):
        response = requests.get(url, stream=True)
        with open(file_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=1024):
                file.write(chunk)
        print(f"Downloaded '{file_path}'.")
    else:
        print(f"File '{file_path}' already exists.")

In [4]:
def unzip(zip_path, extract_to):
    """Unzip a file and delete the .zip file afterward."""
    if zipfile.is_zipfile(zip_path):
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_to)
        print(f"Extracted '{zip_path}' to '{extract_to}'.")
    else:
        print(f"'{zip_path}' is not a valid zip file.")

In [5]:
# Main execution
directory = "datasets"
zip_file_path = os.path.join(directory, "BA_Municipios_2022.zip")
download_url = "https://geoftp.ibge.gov.br/organizacao_do_territorio/malhas_territoriais/malhas_municipais/municipio_2022/UFs/BA/BA_Municipios_2022.zip"  # Replace with the actual URL
directory_out = zip_file_path.split(".")[0]

# Ensure directory exists
ensure_directory_exists(directory)
ensure_directory_exists(directory_out)

# Download the file if it doesn't exist
download_file_if_not_exists(zip_file_path, download_url)

# Unzip the file and delete the .zip
unzip(zip_file_path, directory_out)

Directory 'datasets' already exists.
Directory 'datasets/BA_Municipios_2022' already exists.
File 'datasets/BA_Municipios_2022.zip' already exists.
Extracted 'datasets/BA_Municipios_2022.zip' to 'datasets/BA_Municipios_2022'.


# Load data

In [6]:
import geopandas

In [7]:
data = geopandas.read_file('datasets/BA_Municipios_2022/BA_Municipios_2022.dbf')
data

,CD_MUN,NM_MUN,SIGLA_UF,AREA_KM2,geometry
0,2900108,Abaíra,BA,538.677,"POLYGON ((-41.74667 -13.32509, -41.74679 -13.3..."
1,2900207,Abaré,BA,1604.923,"POLYGON ((-39.23481 -8.70419, -39.23427 -8.705..."
2,2900306,Acajutiba,BA,181.475,"POLYGON ((-38.02809 -11.75007, -38.02811 -11.7..."
3,2900355,Adustina,BA,629.099,"POLYGON ((-37.97867 -10.652, -37.97874 -10.652..."
4,2900405,Água Fria,BA,742.775,"POLYGON ((-38.60169 -11.92432, -38.59707 -11.9..."
...,...,...,...,...,...
412,2933307,Vitória da Conquista,BA,3254.186,"POLYGON ((-40.71779 -14.88926, -40.71774 -14.8..."
413,2933406,Wagner,BA,522.370,"POLYGON ((-41.06276 -12.28494, -41.07128 -12.2..."
414,2933455,Wanderley,BA,2920.579,"POLYGON ((-43.82392 -12.12517, -43.82324 -12.1..."
415,2933505,Wenceslau Guimarães,BA,655.057,"POLYGON ((-39.49336 -13.60308, -39.49335 -13.6..."


In [8]:
data.drop(['NM_MUN','SIGLA_UF'],axis=1,inplace=True)

In [9]:
db_path = 'sql/eleicao.db' 

In [10]:
data.to_file(db_path, driver='SQLite',layer="municipio")